# Lab 4

## Ejercicio 1

In [1]:
import numpy as np

In [4]:
A = np.array([
    [0, 1],
    [-1, 0]
])
B = np.array([
    [-1], 
    [0]
])
U = 1
Q = np.eye(2)
Y1 = 1
Y2 = 2
c = 1
T = 10

In [5]:
from scipy.integrate import solve_ivp

## Ejercicio 2